# LangChain Lab: Exploring the Power of AI

Objectives:
- Install and configure a working environment for LangChain.
- Use an LLM (via LangChain) with prompt templates and chains.
- Explore agents and tools.
- Complete exercises.

## 0) Quick notes before you start

- You will need an OpenAI API key (or another provider) to run the LLM examples.
- Set `OPENAI_API_KEY` in your environment before running cells that call the LLM.
- This notebook uses `langchain` APIs; install the package with the cell below.

## Why these quick notes matter
This block gives important pre-run information and prerequisites. It reminds students to provide an OpenAI API key and install dependencies before running cells that call the LLM; follow these steps to avoid runtime errors.

In [ ]:

%pip install langchain openai langchain-openai langchain-tavily

In [ ]:
import sys
print('Python', sys.version)

## 1) Simple LLM usage with LangChain

introduces basic LangChain interactions: creating prompts, calling an LLM, and reading responses

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = "API_KEY_HERE"

OPENAI_MODEL = "gpt-5-mini"

This commented block demonstrates how to enable detailed logging/debugging for LangChain. Uncomment and run if you need more diagnostics while developing or investigating issues or are curious as to how some of the internals work

In [ ]:
# import logging
# logging.basicConfig(level=logging.INFO)

# from langchain_core.globals import set_debug
# set_debug(True)

In [ ]:
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import OpenAI

prompt_template = "Tell me a {adjective} joke"
prompt = PromptTemplate(
    input_variables=["adjective"], template=prompt_template
)
llm = OpenAI()
chain = prompt | llm | StrOutputParser()

response = chain.invoke("funny")

print(type(chain), type(llm), type(response))
print(response)

## Prompt templates, chains, and simple output parsing
This example demonstrates building a PromptTemplate, connecting it to an OpenAI LLM, and using a simple string output parser. It teaches how to compose small pipelines (prompt → LLM → parser) and call them using `invoke`.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate


llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)

prompt = ChatPromptTemplate.from_template(
    "You are a math tutor. Explain the concept of {topic} clearly."
)

formatted_prompt = prompt.format_messages(topic="Agentic AI")

response = llm.invoke(formatted_prompt)

print(type(llm), type(response))
print(response.content)


## Enforcing output formats — a couple of approaches
This block explains three common ways to get machine-readable, validated output from LLMs:

1) System-prompt / instruction-based enforcement — Add explicit instructions in the system or user prompt asking for a format (e.g., "Return JSON with fields x,y,z"). This is often works, but enforcement is weak (models may deviate). Use this for quick human-readable constraints.

2) PydanticOutputParser (recommended for validation) — Attach a `Pydantic` model and use `PydanticOutputParser.get_format_instructions()` to append precise format instructions to the prompt. After the model returns text, call `parser.parse(...)` to validate and convert into a typed Python object. This gives explicit format instructions and runtime validation (you get errors when output is invalid).

3) `model.with_structured_output(PydanticModel)` — Ask the model wrapper to always return the given Pydantic schema (the SDK handles converting/validating for you). This is convenient when you want the model API to return structured data directly.

Quick notes on trade-offs:
- Instruction-only (system prompt): easiest to write, least safe/validated.
- PydanticOutputParser: explicit instructions + post-hoc validation; good balance of control and visibility.
- with_structured_output: most convenient when supported by the SDK; hides some parsing details.


In [ ]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

class Country(BaseModel):
    capital: str = Field(description="capital of the country")
    name: str = Field(description="name of the country")

PROMPT_COUNTRY_INFO = """
Provide information about {country}.
{format_instructions}
"""

parser = PydanticOutputParser(pydantic_object=Country)
llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)

message = HumanMessagePromptTemplate.from_template(
    template=PROMPT_COUNTRY_INFO,
)
chat_prompt = ChatPromptTemplate.from_messages([message])

# Format the prompt with the country and parser instructions
chat_prompt_with_values = chat_prompt.format_prompt(
    country="Belgium",
    format_instructions=parser.get_format_instructions()
)

print("Input to the model") # This is interesting as we can see how 
for msg in chat_prompt_with_values.messages:
    print(f"{msg.type}: {msg.content}")

output = llm.invoke(chat_prompt_with_values.to_messages())

country = parser.parse(output.content)

print("-----------------------------------")
print("Output of the model: \n")
print(output.content)
print(f"The capital of {country.name} is {country.capital}.")


In [ ]:
from pydantic import BaseModel, Field

model = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)

class Movie(BaseModel):
    """A movie with details."""
    title: str = Field(..., description="The title of the movie")
    year: int = Field(..., description="The year the movie was released")
    director: str = Field(..., description="The director of the movie")
    rating: float = Field(..., description="The movie's rating out of 10")

model_with_structure = model.with_structured_output(Movie)
response = model_with_structure.invoke("Provide details about the movie Inception")
print(response)

## 2) Agents: LLMs + Tools

Agents let a language model orchestrate tools (e.g., python REPL, search, calculator). Below is a minimal demonstration using LangChain's agent framework.

https://docs.langchain.com/oss/python/langchain/agents

## Agents: combining LLMs with tools
Here we introduces agents — systems where an LLM can call external tools (functions) to extend capabilities like search, computation, or side effects. The following cells contain concrete agent examples.

In [ ]:

from langchain.agents import create_agent

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=OPENAI_MODEL,
    tools=[get_weather],
    system_prompt="You are a helpful assistant",
)

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather in sf"}]}
)

print(type(agent))
for msg in response["messages"]:
    msg.pretty_print()



## Direct chat invocation example
This short cell shows calling `ChatOpenAI.invoke` directly with a natural question. Use it to compare direct LLM responses versus agent-enabled behavior (with tools).

This example quickly shows the power of enabling LLMs with tools

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model=OPENAI_MODEL, temperature=0.3)

response = llm.invoke("Who are the professors in Raikes")

print(response.content)


In [ ]:
from langchain.agents import create_agent
import os
from langchain_tavily import TavilySearch

os.environ["TAVILY_API_KEY"] = "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"

# this essentially just allows the llm to search the web, real easy to set up
tavily_search_tool = TavilySearch(
    max_results=2,
    topic="general",
    include_answer=True
)

agent = create_agent(
    model=OPENAI_MODEL,
    tools=[tavily_search_tool],
    system_prompt="You are a helpful assistant",
)

user_input = "Who are the professors in Raikes"
response = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": user_input}
        ]
    }
)

for msg in response["messages"]:
    msg.pretty_print()


## Why agents need tools for math/computation
This cell is an exercise showing a model answering a math question without a calculator tool. It highlights that LLMs can make arithmetic mistakes and motivates adding deterministic computation tools for correctness, highlighting another key use case

In [ ]:

from langchain.agents import create_agent

#TODO: create an addition tool to give the model the ability to answer this question

agent = create_agent(
    model="gpt-3.5-turbo",
    tools=[],
    system_prompt="You are a helpful assistant",
)

user_input = "What is 13781 times 2394" # answer should be 32991714, model should get this wrong without a tool
response = agent.invoke(
    {"messages": [{"role": "user", "content": user_input}]}
)

for msg in response["messages"]:
        msg.pretty_print()



## Challange: Write your own tool
This template shows the shape a tool should take (typed arguments and a clear docstring). The model uses the function signature and docs to know what arguments to pass — implement the body to provide useful functionality.

Attempt to create a tool to remedy something you have noticed that LLMs struggle with. If you aren't able to implement it in the time we have, that's fine, it is the insight that matters.

In [ ]:

from langchain.agents import create_agent

def your_tool(foo: str, bar: int) -> str:
    """
        Write the function docs here. They are important!
        This is how the model knows what the function does
        Make sure to include the types in the function definition.
        That is how the model knows what to pass in as arguements
    """
    # TODO: Implement tool
    return None

agent = create_agent(
    model=OPENAI_MODEL,
    tools=[your_tool],
    system_prompt="You are a helpful assistant",
)

user_input = "ask the model something here"
response = agent.invoke(
    {"messages": [{"role": "user", "content": user_input}]}
)

for msg in response["messages"]:
    msg.pretty_print()



## Structured tool outputs and response schemas

This example demonstrates how to return structured data from an agent using `ToolStrategy` or `ProviderStrategy` with a Pydantic model.

By defining explicit response schemas, agents can produce consistent, machine-readable outputs rather than free-form text. This approach is valuable for a wide range of applications, such as:

- **Data extraction** — pulling specific fields or entities from unstructured input.  
- **API integration** — ensuring responses match required request/response formats.  
- **Form filling and automation** — mapping model outputs directly into databases or UI forms.  
- **Validation and error handling** — enforcing data types and constraints before downstream use.  
- **Complex workflows** — passing structured intermediate results between tools or agents.

Structured output schemas improve reliability, enable automatic validation, and make it easier to compose AI components into larger, predictable systems.


In [5]:
from pydantic import BaseModel
from langchain.agents import create_agent
from langchain.agents.structured_output import ToolStrategy
from langchain.agents.structured_output import ProviderStrategy


class ContactInfo(BaseModel):
    name: str
    email: str
    phone: str

agent = create_agent(
    model=OPENAI_MODEL,
    tools=[],
    response_format=ToolStrategy(ContactInfo) #Change ToolStrategy with ProviderStrategy to see how output differs
)

result = agent.invoke({
    "messages": [{"role": "user", "content": "Extract contact info from: John Doe, john@example.com, (555) 123-4567"}]
})

for msg in result["messages"]:
    msg.pretty_print()

print("\n\nStructured response from agent:")
print(result["structured_response"])


================================ Human Message =================================

Extract contact info from: John Doe, john@example.com, (555) 123-4567
================================== Ai Message ==================================
Tool Calls:
  ContactInfo (call_Be7KUhSPuwOTAomLY3TgChNN)
 Call ID: call_Be7KUhSPuwOTAomLY3TgChNN
  Args:
    name: John Doe
    email: john@example.com
    phone: (555) 123-4567
================================= Tool Message =================================
Name: ContactInfo

Returning structured response: name='John Doe' email='john@example.com' phone='(555) 123-4567'


Structured response from agent:
name='John Doe' email='john@example.com' phone='(555) 123-4567'


## Human-in-the-loop approval and tool execution
This longer example shows middleware that prompts a human to approve or edit a tool call before execution. 

This example implements a checkpointer which allows us to track and come back to specific sessions as indicated by the thread_id.

This Human-in-the-loop architecture is key for ensuring safe workflows where humans can limit the negative side effects of AI.

Areas where this is applicable include:
- **Financial decision-making**, where transactions or investments must be reviewed before commitment.  
- **Healthcare applications**, where AI suggestions should be validated by medical professionals.  
- **Data modification or deletion tasks**, where human approval prevents accidental or malicious actions.  
- **Autonomous systems or robotics**, where human oversight ensures safety in physical environments.  
- **Customer service and compliance**, where humans can approve sensitive responses or verify legal constraints.


In [ ]:
from langchain.agents import create_agent
from langchain_openai import ChatOpenAI
from langchain.agents.middleware import HumanInTheLoopMiddleware
from langgraph.checkpoint.memory import InMemorySaver

# === Mock tools ===
def send_email_tool(recipient: str, subject: str, body: str) -> str:
    """Pretend to send an email."""
    return f"✅ Email sent to {recipient} with subject '{subject}'."

def read_email_tool() -> str:
    """Pretend to read an email."""
    return "📧 You have 2 unread emails from Alice and Bob."

tools = [send_email_tool, read_email_tool]

# === Create the agent ===
agent = create_agent(
    model=OPENAI_MODEL,
    tools=tools,
    checkpointer=InMemorySaver(),
    middleware=[
        HumanInTheLoopMiddleware(
            interrupt_on={
                "send_email_tool": {
                    "allowed_decisions": ["approve", "edit", "reject"],
                },
                "read_email_tool": False,  # auto-approve reads
            }
        ),
    ],
)

import random
thread_id = random.randint(1, 1000000)

result = agent.invoke({
    "messages": [
        {"role": "user", "content": "Send an email to alice@example.com saying I’ll be late to the meeting."}
    ]
}, {"configurable": {"thread_id": thread_id}})

last_msg = result["messages"][-1]
if hasattr(last_msg, "tool_calls") and last_msg.tool_calls:
    tool_call = last_msg.tool_calls[0]
    tool_name = tool_call["name"]
    tool_args = tool_call["args"]

    print(f"\nProposed tool call: {tool_name}")
    for k, v in tool_args.items():
        print(f"  {k}: {v}")

    decision = input("\nApprove or reject? ").strip().lower()

    if decision == "approve":
        tool_func = next(t for t in tools if t.__name__ == tool_name)
        output = tool_func(**tool_args)
        followup = agent.invoke({
            "messages": [
                *result["messages"],
                {"role": "tool", "name": tool_name, "content": output, "tool_call_id": tool_call["id"]},
            ]
        }, {"configurable": {"thread_id": thread_id}})
        for msg in followup["messages"]:
            msg.pretty_print()
    else:
        print("❌ Tool call rejected.")

else:
    for msg in result["messages"]:
        msg.pretty_print()


## CHALLENGE: Create your own agent

- Try giving it a system prompt that seriously affects how it responds to prompts
- Give it a couple tools to see when and how it uses them
- Try something you were always curious about

In [ ]:
#TODO: Create your own agent